In [48]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
from scipy import stats
from statsmodels.stats.power import tt_ind_solve_power

# Кейс

Монетизация классифайда. Проведен тест с экранами платных услуг в монетизации. Гипотеза - можно увеличить ARPU на +3.5 рубля.

# Загрузка данных

Схема данных
- ARPU: целевая метрика в тесте
- is_capital: продавец авто из столицы (capital) или из регионов (region)
- is_pro: профессиональный или нет продавец (1 — профи, 0 — простой)
- group: группа в А/Б-тесте (1 — тестовая, 0 — контрольная)

In [50]:
df_stratification = pd.read_csv('stratification.csv')
df_stratification.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9700 entries, 0 to 9699
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ARPU        9700 non-null   float64
 1   is_capital  9700 non-null   object 
 2   is_pro      9700 non-null   int64  
 3   group       9700 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 303.3+ KB


In [7]:
df_stratification.head()

,ARPU,is_capital,is_pro,group
0,250.5,region,1,1
1,182.0,capital,0,1
2,75.0,region,0,0
3,532.5,capital,1,0
4,88.0,region,0,1


# Стратификация

## T-test

In [13]:
def get_basic_ttest(group_A, group_B):
    '''Проверяет гипотезу о равенстве средних для обычного среднего.
    return - t_stat, p_value.'''

    t_stat, p_value = ttest_ind(group_A, group_B)
    inference = {'t_stat': t_stat, 'p_value':p_value}
    return(inference)

In [15]:
inference = get_basic_ttest(df_stratification['ARPU'][df_stratification['group']==0], df_stratification['ARPU'][df_stratification['group']==1])
print('p-value базового t-test = ', round(inference['p_value'], 3))

p-value базового t-test =  0.065


## Стратифицированная диспесия

In [18]:
def calculate_strat_var(data, strata_name, gen_pop_weights, target_value):
    '''Вычисляет стратифицированную дисперсию.'''
    
    strat_vars = data.groupby(strata_name)[target_value].var()
    
    data_vars_weights = pd.merge(
        pd.Series(strat_vars, name='value_vars'),
        pd.Series(gen_pop_weights, name='weight'),
        how='inner',
        left_index=True,
        right_index=True)   
 
    var_strat = (data_vars_weights['weight'] * data_vars_weights['value_vars']).sum()
    return (var_strat)

In [20]:
gen_var = round(df_stratification['ARPU'].var())
print('Глобальная дисперсия данных', gen_var)

Глобальная дисперсия данных 7480


### is_capital

In [23]:
# Рассчитаем доли генеральной совокупности по is_capital
gen_pop_weights_cap = df_stratification['is_capital'].value_counts(normalize=True).to_dict()

# Вычислим стратифицированную дисперсию
strat_var_cap = calculate_strat_var(df_stratification, 'is_capital', gen_pop_weights_cap, 'ARPU')
disp_decr = round(strat_var_cap/gen_var - 1, 3)

print(f'Стратифицированная дисперсия по is_capital: {strat_var_cap:.4f}')
print('Дисперсия сокращается на {}%'.format(disp_decr*100))

Стратифицированная дисперсия по is_capital: 5789.9483
Дисперсия сокращается на -22.6%


### is_pro

In [26]:
# Рассчитаем доли генеральной совокупности по is_pro
gen_pop_weights_pro = df_stratification['is_pro'].value_counts(normalize=True).to_dict()

# Вычислим стратифицированную дисперсию
strat_var_pro = calculate_strat_var(df_stratification, 'is_pro', gen_pop_weights_pro, 'ARPU')
disp_decr = round(strat_var_pro/gen_var - 1, 3)

print(f'Стратифицированная дисперсия по is_pro: {strat_var_pro:.4f}')
print('Дисперсия сокращается на {}%'.format(disp_decr*100))

Стратифицированная дисперсия по is_pro: 6193.9351
Дисперсия сокращается на -17.2%


Дисперсия сокращается максимально при страте is_capital

# Постстратификация

In [30]:
# Функция для расчета стратифицированного среднего
def calculate_strat_mean(data, strata_name, gen_pop_weights, target_value):
    strats_means = data.groupby(strata_name)[target_value].mean()
    data_means_weights = pd.merge(
        pd.Series(strats_means, name='value_means'),
        pd.Series(gen_pop_weights, name='weight'),
        how='inner',
        left_index=True,
        right_index=True)
    mean_strat = (data_means_weights['weight'] * data_means_weights['value_means']).sum()
    return mean_strat

In [32]:
# Функция для расчета t-теста с учетом стратификации
def get_strat_ttest(df_A, df_B, strata_name, target_value, gen_pop_weights):
    # Расчет стратифицированных средних для обеих групп
    mean_strat_A = calculate_strat_mean(df_A, strata_name, gen_pop_weights, target_value)
    mean_strat_B = calculate_strat_mean(df_B, strata_name, gen_pop_weights, target_value)
    
    # Расчет стратифицированной дисперсии для обеих групп
    var_strat_A = calculate_strat_var(df_A, strata_name, gen_pop_weights, target_value)
    var_strat_B = calculate_strat_var(df_B, strata_name, gen_pop_weights, target_value)
    
    # Расчет статистики t
    delta_mean_strat = mean_strat_A - mean_strat_B
    std_mean_strat = np.sqrt(var_strat_A / len(df_A) + var_strat_B / len(df_B))
    t_stat_strat = delta_mean_strat / std_mean_strat
    
    # Расчет p-value
    p_value = 2 * (1 - stats.norm.cdf(np.abs(t_stat_strat)))
    
    # Возврат результатов
    inference = {'t_stat': t_stat_strat, 'p_value': p_value}
    return inference

In [34]:
# Находим веса для генеральной совокупности по стратифицированной переменной
gen_pop_weights = df_stratification['is_capital'].value_counts(normalize=True).to_dict()

# Разделим данные на тестовую и контрольную группы
df_A = df_stratification[df_stratification['group'] == 1]
df_B = df_stratification[df_stratification['group'] == 0]

# Применим постстратификацию и пересчитаем t-test
result = get_strat_ttest(df_A, df_B, 'is_capital', 'ARPU', gen_pop_weights)

print(f"T-statistic: {result['t_stat']:.4f}")
print(f"P-value: {result['p_value']:.4f}")

T-statistic: 2.3578
P-value: 0.0184


# Размер выборки

In [44]:
#зададим исходные распределения
abs_MDE = df_stratification['ARPU'][df_stratification['group']==1].mean() - df_stratification['ARPU'][df_stratification['group']==0].mean()
print('Абсолютный MDE =', abs_MDE)

# работаем только с группой A
data_a = df_stratification[df_stratification['group']==0]

basic_std = data_a['ARPU'].std()
basic_cohen_D_effect_size = abs_MDE / basic_std
basic_sample_size = int(tt_ind_solve_power(effect_size = basic_cohen_D_effect_size, alpha=0.05, power=0.8, nobs1=None, ratio=1))

Абсолютный MDE = 3.2421437067065995


In [46]:
# работаем только с группой A
group_a_weights = dict(data_a['is_capital'].value_counts(normalize = True))

# рассчитаем стратифицированное стандартное отклонение
strat_std = calculate_strat_var(data_a, 'is_capital', group_a_weights, 'ARPU') ** 0.5

print('Обычная станд. отклонение =', basic_std)
print('Стратифицированное станд. отклонение =', strat_std)

basic_cohen_D_effect_size = abs_MDE / basic_std
strat_cohen_D_effect_size = abs_MDE / strat_std

print('basic Cohen D effect size = ', basic_cohen_D_effect_size)
print('strat Cohen D effect size = ', strat_cohen_D_effect_size)

strat_sample_size = int(tt_ind_solve_power(effect_size = strat_cohen_D_effect_size, alpha=0.05, power=0.8, nobs1=None, ratio=1))
basic_sample_size = int(tt_ind_solve_power(effect_size = basic_cohen_D_effect_size, alpha=0.05, power=0.8, nobs1=None, ratio=1))
print('Ответ--------------------------------')
print('Basic sample size = {}'.format(basic_sample_size))
print('Stratified sample size = {}'.format(strat_sample_size))

Обычная станд. отклонение = 87.24597419606758
Стратифицированное станд. отклонение = 76.1470117161366
basic Cohen D effect size =  0.0371609548358133
strat Cohen D effect size =  0.042577425346549014
Ответ--------------------------------
Basic sample size = 11368
Stratified sample size = 8660
